# Segment clouds and cloud shadows in Landsat-8 images (L1C)
This notebook shows an example on how to use [ukis-csmask](https://github.com/dlr-eoc/ukis-csmask) to segment clouds and cloud shadows in Level-1C images from Landsat-8. Images are loaded from local file system. Here we use [ukis-pysat](https://github.com/dlr-eoc/ukis-pysat) for convencience image handling, but you can also work directly with [numpy](https://numpy.org/) arrays.

> NOTE: to run this notebook, we first need to install some additional dependencies for image handling
```shell
pip install ukis-pysat[complete]
```

In [ ]:
import rasterio
import numpy as np

from pathlib import Path
from ukis_csmask.mask import CSmask
from ukis_pysat.raster import Image, Platform

In [ ]:
# user settings
data_path = "/your_data_path/"
L8_file_prefix = "LC08_L1TP_191015_20210428_20210507_02_T1"
product_level = "l1c"
band_order = ["blue", "green", "red", "nir", "swir16", "swir22"]
providers = ["CUDAExecutionProvider"]
out_dir = "ukis-csmask/examples"

In [ ]:
# set Landsat 8 source path and prefix (example)
data_path = data_path + L8_file_prefix + "/"
mtl_file = data_path + L8_file_prefix + "_MTL.txt"

# stack [B2:'Blue', B3:'Green', B4:'Red', B5:'NIR', B6:'SWIR1', B7:'SWIR2'] as numpy array
L8_band_files = [data_path + L8_file_prefix + "_B" + x + ".TIF" for x in [str(x + 2) for x in range(6)]]

# >> adopted from https://gis.stackexchange.com/questions/223910/using-rasterio-or-gdal-to-stack-multiple-bands-without-using-subprocess-commands
# read metadata of first file
with rasterio.open(L8_band_files[0]) as src0:
    meta = src0.meta
# update meta to reflect the number of layers
meta.update(count=len(L8_band_files))
# read each layer and append it to numpy array
L8_bands = []
for id, layer in enumerate(L8_band_files, start=1):
    with rasterio.open(layer) as src1:
        L8_bands.append(src1.read(1))
L8_bands = np.stack(L8_bands, axis=2)
# <<

img = Image(data=L8_bands, crs=meta["crs"], transform=meta["transform"], dimorder="last")
img.dn2toa(platform=Platform.Landsat8, mtl_file=mtl_file, wavelengths=band_order)
img.warp(resampling_method=0, resolution=30, dst_crs=img.dataset.crs)

In [ ]:
# compute cloud and cloud shadow mask
csmask = CSmask(
    img=img.arr,
    band_order=band_order,
    product_level=product_level,
    nodata_value=0,
    invalid_buffer=4,
    intra_op_num_threads=0,
    inter_op_num_threads=0,
    providers=providers,
    batch_size=1,
)

# access cloud and cloud shadow mask as numpy array
csm = csmask.csm

# access valid mask as numpy array
valid = csmask.valid

In [ ]:
# convert results to ukis-pysat Image
# this assigns back the georeference
csm = Image(csm, transform=img.dataset.transform, crs=img.dataset.crs, dimorder="last")
valid = Image(valid, transform=img.dataset.transform, crs=img.dataset.crs, dimorder="last")

# write results to file
csm.write_to_file(
    path_to_file=Path(out_dir) / Path(f"{L8_file_prefix}_csm.tif"),
    dtype=csm.dtype,
    driver="COG",
    compress="LZW",
    kwargs={"BLOCKSIZE": 512, "BIGTIFF": "IF_SAFER"},
)
valid.write_to_file(
    path_to_file=Path(out_dir) / Path(f"{L8_file_prefix}_valid.tif"),
    dtype=valid.dtype,
    driver="COG",
    compress="LZW",
    kwargs={"BLOCKSIZE": 512, "BIGTIFF": "IF_SAFER"},
)